In [2]:
# %matplotlib notebook
# %load_ext autoreload
# %autoreload 2

import os
import random

from operator import itemgetter

import cv2
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

from albumentations import (
    CLAHE,
    Blur,
    OneOf,
    Compose,
    RGBShift,
    GaussNoise,
    RandomGamma,
    RandomContrast,
    RandomBrightness,
)

from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.transforms import HueSaturationValue
from albumentations.augmentations.transforms import Normalize

from trainer import Trainer, hooks, configuration
from detector import Detector
from trainer.utils import patch_configs
from trainer.utils import setup_system
from detection_loss import DetectionLoss
from trainer.encoder import (
    DataEncoder,
    decode_boxes,
    encode_boxes,
    generate_anchors,
    generate_anchor_grid,
)
from trainer.metrics import APEstimator
from trainer.datasets import ListDataset
# from trainer.data_set_downloader import DataSetDownloader 
from trainer.matplotlib_visualizer import MatplotlibVisualizer

from typing import Callable, Iterable
from dataclasses import dataclass

from torchvision.transforms import ToTensor

/home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
@dataclass
class SystemConfig:
    seed: int = 42  # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = False  # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True  # make cudnn deterministic (reproducible training)


@dataclass
class DatasetConfig:
    root_dir: str = "data"  # dataset directory root
    train_transforms: Iterable[Callable] = (
        ToTensor(),
    )  # data transformation to use during training data preparation
    test_transforms: Iterable[Callable] = (
        ToTensor(),
    )  # data transformation to use during test data preparation


@dataclass
class DataloaderConfig:
    batch_size: int = 250  # amount of data to pass through the network at each forward-backward iteration
    num_workers: int = 5  # number of concurrent processes using to prepare data


@dataclass
class OptimizerConfig:
    learning_rate: float = 0.001  # determines the speed of network's weights update
    momentum: float = 0.9  # used to improve vanilla SGD algorithm and provide better handling of local minimas
    weight_decay: float = 0.0001  # amount of additional regularization on the weights values
    lr_step_milestones: Iterable = (
        30, 40
    )  # at which epoches should we make a "step" in learning rate (i.e. decrease it in some manner)
    lr_gamma: float = 0.1  # multiplier applied to current learning rate at each of lr_ctep_milestones


@dataclass
class TrainerConfig:
    model_dir: str = "checkpoints"  # directory to save model states
    model_save_best: bool = True  # save model with best accuracy
    model_saving_frequency: int = 1  # frequency of model state savings per epochs
    device: str = "cpu"  # device to use for training.
    epoch_num: int = 50  # number of times the whole dataset will be passed through the network
    progress_bar: bool = False  # enable progress bar visualization during train process


In [4]:
def patch_configs(epoch_num_to_set=TrainerConfig.epoch_num, batch_size_to_set=DataloaderConfig.batch_size):
    """ Patches configs if cuda is not available

    Returns:
        returns patched dataloader_config and trainer_config

    """
    # default experiment params
    num_workers_to_set = DataloaderConfig.num_workers

    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
        batch_size_to_set = 16
        num_workers_to_set = 2
        epoch_num_to_set = 1

    dataloader_config = DataloaderConfig(batch_size=batch_size_to_set, num_workers=num_workers_to_set)
    trainer_config = TrainerConfig(device=device, epoch_num=epoch_num_to_set, progress_bar=True)

In [5]:
dataloader_config, trainer_config = patch_configs(epoch_num_to_set=10, batch_size_to_set=1)
dataset_config = configuration.DatasetConfig(
    root_dir="../../../Datasets/Road_Scene_Object_Detection",
    train_transforms=[
        RandomBrightness(p=0.5),
        RandomContrast(p=0.5),
        OneOf([
            RandomGamma(),
            HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50),
            RGBShift()
        ],
            p=1),
        OneOf([Blur(always_apply=True), GaussNoise(always_apply=True)], p=1),
        CLAHE(),
        Normalize(),
        ToTensorV2()
    ]
)

TypeError: 'NoneType' object is not iterable

In [4]:

dataset_train = ListDataset(
    root_dir=dataset_config.root_dir,
    data_dir = 'export',
    list_file='annots_converted_train.txt',

    classes=[
        "__background__",
        "biker",
        "car",
        "pedestrian",
        "trafficLight",
        "trafficLight-Green",
        "trafficLight-GreenLeft",
        "trafficLight-Red",
        "trafficLight-RedLeft",
        "trafficLight-Yellow",
        "trafficLight-YellowLeft",
        "truck"
    ],
    mode='train',
    transform=Compose(dataset_config.train_transforms),
    input_size=300
)

loader_train = DataLoader(
    dataset=dataset_train,
    batch_size=dataloader_config.batch_size,
    shuffle=True,
#     collate_fn=dataset_train.collate_fn,
    num_workers=dataloader_config.num_workers,
    pin_memory=True
)

In [5]:
iterator = iter(loader_train)
one_batch = next(iterator)

In [6]:
len(one_batch)

3

In [7]:
one_batch = next(iterator)

In [8]:
one_batch[1].shape

torch.Size([1, 17451, 4])

In [9]:
one_batch[2].shape

torch.Size([1, 17451])

In [10]:
one_batch[2]

tensor([[0, 0, 0,  ..., 0, 0, 0]])